In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import random
import numpy as np
import torch
import math

In [3]:
from PIL import Image, ImageOps
from argparse import ArgumentParser

from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage

In [4]:
import importlib
from iouEval import iouEval, getColorEntry

In [5]:
from dataset import lanes
from transform import Relabel, Relabel_enc, ToLabel, Colorize
from visualize import Dashboard
from shutil import copyfile

In [6]:
NUM_CHANNELS = 3
NUM_CLASSES = 3 #pascal=22, cityscapes=20

color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()

In [7]:
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        targets1 = Resize((self.height, 904), Image.NEAREST)(target[0])
        targets2 = Resize((self.height, 904), Image.NEAREST)(target[1])
        targets3 = Resize((self.height, 904), Image.NEAREST)(target[2])

        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                targets1 = targets1.transpose(Image.FLIP_LEFT_RIGHT)
                targets2 = targets2.transpose(Image.FLIP_LEFT_RIGHT)
                targets3 = targets3.transpose(Image.FLIP_LEFT_RIGHT)
            
            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2) 
            transY = random.randint(-2, 2)

            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            targets1 = ImageOps.expand(targets1, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets2 = ImageOps.expand(targets2, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            targets3 = ImageOps.expand(targets3, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            targets1 = targets1.crop((0, 0, targets1.size[0]-transX, targets1.size[1]-transY))
            targets2 = targets2.crop((0, 0, targets2.size[0]-transX, targets2.size[1]-transY))
            targets3 = targets3.crop((0, 0, targets3.size[0]-transX, targets3.size[1]-transY))

        input = ToTensor()(input)
        if (self.enc):
            targets1 = Resize(int(self.height/8), Image.NEAREST)(targets1)
            targets2 = Resize(int(self.height/8), Image.NEAREST)(targets2)
            targets3 = Resize(int(self.height/8), Image.NEAREST)(targets3)
        target = np.zeros_like(targets1)
        targets1, targets2, targets3, target = ToLabel()(targets1), ToLabel()(targets2), ToLabel()(targets3), ToLabel()(target)
        return input, Relabel_enc()(targets1, targets2, targets3, target)

In [8]:
class CrossEntropyLoss2d(torch.nn.Module):

    def __init__(self, weight=None):
        super().__init__()

        self.loss = torch.nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)

In [9]:
def train(model, enc=False, cuda=True, train_datadir="train_data_3/", test_datadir="test_data_3/", resume=False, num_epochs=250,
         visualize=False, steps_plot=50, port=8888, iouTrain=False, iouVal=True, height=512, num_workers=0, batch_size=4,
         steps_loss=50, epochs_save=0):
    best_acc = 0

    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 50
        weight[1] = 50
    
    else:
        weight[0] = 50
        weight[1] = 50
    weight[2] = 0

    assert os.path.exists(train_datadir), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=height)#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=height)#1024)
    dataset_train = lanes(train_datadir, co_transform)
    dataset_val = lanes(test_datadir, co_transform_val)

    loader = DataLoader(dataset_train, num_workers=num_workers, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=num_workers, batch_size=batch_size, shuffle=False)

    if cuda:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = f'../save/'

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"    

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists 
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))


    #TODO: reduce memory in first gpu: https://discuss.pytorch.org/t/multi-gpu-training-memory-usage-in-balance/4163/4        #https://github.com/pytorch/pytorch/issues/1893

    #optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=2e-4)     ## scheduler 1
    optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)      ## scheduler 2

    start_epoch = 1
    if resume:
        #Must load weights, optimizer, epoch and best value. 
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/num_epochs)),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if visualize and steps_plot > 0:
        board = Dashboard(port)

    for epoch in range(start_epoch, num_epochs+1):
        print("----- TRAINING - EPOCH", epoch, "-----")

        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
     
        doIouTrain = iouTrain   
        doIouVal =  iouVal      

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()

        for step, (images, labels) in enumerate(loader):

            #print (labels.size())
            #print (np.unique(labels.numpy()))
            #print("labels: ", np.unique(labels[0].numpy()))
            #labels = torch.ones(4, 1, 512, 1024).long()
            if cuda:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs, only_encode=enc)

            #print("targets", np.unique(targets[:, 0].cpu().data.numpy()))

            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()
            

            epoch_loss.append(loss.data)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)    

            #print(outputs.size())
            if visualize and steps_plot > 0 and step % steps_plot == 0:
                image = inputs[0].cpu().data
                #image[0] = image[0] * .229 + .485
                #image[1] = image[1] * .224 + .456
                #image[2] = image[2] * .225 + .406
                #print("output", np.unique(outputs[0].cpu().max(0)[1].data.numpy()))
                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
            if steps_loss > 0 and step % steps_loss == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})')

            
        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)
        
        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")  

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        with torch.no_grad():
            for step, (images, labels) in enumerate(loader_val):
                if cuda:
                    images = images.cuda()
                    labels = labels.cuda()

                inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
                targets = Variable(labels, volatile=True)
                outputs = model(inputs, only_encode=enc) 
                
                loss = criterion(outputs, targets[:, 0])
                epoch_loss_val.append(loss.data)


                #Add batch to calculate TP, FP and FN for iou estimation
                if (doIouVal):
                    iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

                if visualize and steps_plot > 0 and step % steps_plot == 0:
                    image = inputs[0].cpu().data
                    board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                    if isinstance(outputs, list):   #merge gpu tensors
                        board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                        f'VAL output (epoch: {epoch}, step: {step})')
                    else:
                        board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                        f'VAL output (epoch: {epoch}, step: {step})')
                    board.image(color_transform(targets[0].cpu().data),
                        f'VAL target (epoch: {epoch}, step: {step})')
                if steps_loss > 0 and step % steps_loss == 0:
                    average = sum(epoch_loss_val) / len(epoch_loss_val)
                    print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})')
                       

        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)
        #scheduler.step(average_epoch_loss_val, epoch)  ## scheduler 1   # update lr if needed

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%") 
           

        # remember best valIoU and save checkpoint
        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal 
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'    
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if epochs_save > 0 and step > 0 and step % epochs_save == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))   
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))           

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))
    
    return(model)   #return model (convenience for encoder-decoder training)


In [10]:
def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

In [11]:
def main(modell, decoder=False, state=False, pretrainedEncoder=False, cuda=True):
    savedir = f'../save/'

    if not os.path.exists(savedir):
        os.makedirs(savedir)

    assert os.path.exists(modell + ".py"), "Error: model definition not found"
    model_file = importlib.import_module(modell)
    model = model_file.Net(NUM_CLASSES)
    copyfile(modell + ".py", savedir + '/' + modell + ".py")
    
    if cuda:
        model = torch.nn.DataParallel(model).cuda()
    
    if state:
        #if args.state is provided then load this state for training
        #Note: this only loads initialized weights. If you want to resume a training use "--resume" option!!
        """
        try:
            model.load_state_dict(torch.load(args.state))
        except AssertionError:
            model.load_state_dict(torch.load(args.state,
                map_location=lambda storage, loc: storage))
        #When model is saved as DataParallel it adds a model. to each key. To remove:
        #state_dict = {k.partition('model.')[2]: v for k,v in state_dict}
        #https://discuss.pytorch.org/t/prefix-parameter-names-in-saved-model-if-trained-by-multi-gpu/494
        """
        def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict keys are there
            own_state = model.state_dict()
            for name, param in state_dict.items():
                if name not in own_state:
                     continue
                own_state[name].copy_(param)
            return model

        #print(torch.load(args.state))
        #model = load_my_state_dict(model, torch.load(state))
        model.load_state_dict(torch.load("../save/model_best.pth"))

    #train(args, model)
    if (not decoder):
        print("========== ENCODER TRAINING ===========")
        model = train(model, True) #Train encoder
    #CAREFUL: for some reason, after training encoder alone, the decoder gets weights=0. 
    #We must reinit decoder weights or reload network passing only encoder in order to train decoder
    print("========== DECODER TRAINING ===========")
    if (not state):
        if pretrainedEncoder:
            print("Loading encoder pretrained in imagenet")
            from erfnet_imagenet import ERFNet as ERFNet_imagenet
            pretrainedEnc = torch.nn.DataParallel(ERFNet_imagenet(1000))
            pretrainedEnc.load_state_dict(torch.load(pretrainedEncoder)['state_dict'])
            pretrainedEnc = next(pretrainedEnc.children()).features.encoder
            if (not cuda):
                pretrainedEnc = pretrainedEnc.cpu()     #because loaded encoder is probably saved in cuda
        else:
            pretrainedEnc = next(model.children()).encoder
        model = model_file.Net(NUM_CLASSES, encoder=pretrainedEnc)  #Add decoder to encoder
        if cuda:
            model = torch.nn.DataParallel(model).cuda()
        #When loading encoder reinitialize weights for decoder because they are set to 0 when training dec
    model = train(model, enc=False)   #Train decoder
    print("========== TRAINING FINISHED ===========")


In [12]:
main(modell="erfnet")

========== ENCODER TRAINING ===========
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005
loss: 1.61 (epoch: 1, step: 0)
loss: 0.1379 (epoch: 1, step: 50)
loss: 0.08468 (epoch: 1, step: 100)
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.02616 (epoch: 1, step: 0)
VAL loss: 0.02592 (epoch: 1, step: 50)
EPOCH IoU on VAL set:  34.80 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.0004981996394708885
loss: 0.02627 (epoch: 2, step: 0)
loss: 0.02395 (epoch: 2, step: 50)
loss: 0.02315 (epoch: 2, step: 100)
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.02257 (epoch: 2, step: 0)
VAL loss: 0.02148 (epoch: 2, step: 50)
EPOCH IoU on VAL set:  36.36 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 2)
----- TRAINING - EPOCH 3 -----
LEARNING RATE:  0.0004963985557581707
loss: 0.02579 (epoch: 3, step: 0)
loss: 0.02 (epoch: 3, step: 50)
loss: 0.0194 (epoch: 3, step: 100)
-

loss: 0.009801 (epoch: 24, step: 0)
loss: 0.01231 (epoch: 24, step: 50)
loss: 0.01267 (epoch: 24, step: 100)
----- VALIDATING - EPOCH 24 -----
VAL loss: 0.01323 (epoch: 24, step: 0)
VAL loss: 0.01317 (epoch: 24, step: 50)
EPOCH IoU on VAL set:  39.80 %
----- TRAINING - EPOCH 25 -----
LEARNING RATE:  0.0004565849496141698
loss: 0.01476 (epoch: 25, step: 0)
loss: 0.01227 (epoch: 25, step: 50)
loss: 0.01232 (epoch: 25, step: 100)
----- VALIDATING - EPOCH 25 -----
VAL loss: 0.01574 (epoch: 25, step: 0)
VAL loss: 0.01267 (epoch: 25, step: 50)
EPOCH IoU on VAL set:  42.40 %
----- TRAINING - EPOCH 26 -----
LEARNING RATE:  0.00045476628804148113
loss: 0.01806 (epoch: 26, step: 0)
loss: 0.01242 (epoch: 26, step: 50)
loss: 0.01266 (epoch: 26, step: 100)
----- VALIDATING - EPOCH 26 -----
VAL loss: 0.01507 (epoch: 26, step: 0)
VAL loss: 0.01314 (epoch: 26, step: 50)
EPOCH IoU on VAL set:  42.63 %
----- TRAINING - EPOCH 27 -----
LEARNING RATE:  0.00045294681799231724
loss: 0.01329 (epoch: 27, step:

loss: 0.01197 (epoch: 48, step: 100)
----- VALIDATING - EPOCH 48 -----
VAL loss: 0.01584 (epoch: 48, step: 0)
VAL loss: 0.01403 (epoch: 48, step: 50)
EPOCH IoU on VAL set:  41.83 %
----- TRAINING - EPOCH 49 -----
LEARNING RATE:  0.0004127054737790459
loss: 0.0128 (epoch: 49, step: 0)
loss: 0.01163 (epoch: 49, step: 50)
loss: 0.01173 (epoch: 49, step: 100)
----- VALIDATING - EPOCH 49 -----
VAL loss: 0.01097 (epoch: 49, step: 0)
VAL loss: 0.01151 (epoch: 49, step: 50)
EPOCH IoU on VAL set:  43.54 %
----- TRAINING - EPOCH 50 -----
LEARNING RATE:  0.00041086623104112007
loss: 0.01135 (epoch: 50, step: 0)
loss: 0.01183 (epoch: 50, step: 50)
loss: 0.01185 (epoch: 50, step: 100)
----- VALIDATING - EPOCH 50 -----
VAL loss: 0.0141 (epoch: 50, step: 0)
VAL loss: 0.01216 (epoch: 50, step: 50)
EPOCH IoU on VAL set:  41.88 %
----- TRAINING - EPOCH 51 -----
LEARNING RATE:  0.00040902607302542923
loss: 0.01082 (epoch: 51, step: 0)
loss: 0.01199 (epoch: 51, step: 50)
loss: 0.01165 (epoch: 51, step: 10

loss: 0.01112 (epoch: 72, step: 100)
----- VALIDATING - EPOCH 72 -----
VAL loss: 0.01298 (epoch: 72, step: 0)
VAL loss: 0.01041 (epoch: 72, step: 50)
EPOCH IoU on VAL set:  46.00 %
Saving model as best
save: ../save//model_encoder_best.pth (epoch: 72)
----- TRAINING - EPOCH 73 -----
LEARNING RATE:  0.00036830023724347775
loss: 0.009208 (epoch: 73, step: 0)
loss: 0.01096 (epoch: 73, step: 50)
loss: 0.01097 (epoch: 73, step: 100)
----- VALIDATING - EPOCH 73 -----
VAL loss: 0.01229 (epoch: 73, step: 0)
VAL loss: 0.01087 (epoch: 73, step: 50)
EPOCH IoU on VAL set:  43.69 %
----- TRAINING - EPOCH 74 -----
LEARNING RATE:  0.0003664375208642858
loss: 0.01004 (epoch: 74, step: 0)
loss: 0.01093 (epoch: 74, step: 50)
loss: 0.01104 (epoch: 74, step: 100)
----- VALIDATING - EPOCH 74 -----
VAL loss: 0.01285 (epoch: 74, step: 0)
VAL loss: 0.01237 (epoch: 74, step: 50)
EPOCH IoU on VAL set:  43.52 %
----- TRAINING - EPOCH 75 -----
LEARNING RATE:  0.00036457375179973773
loss: 0.01012 (epoch: 75, step:

VAL loss: 0.01042 (epoch: 96, step: 50)
EPOCH IoU on VAL set:  45.53 %
----- TRAINING - EPOCH 97 -----
LEARNING RATE:  0.0003232902784463182
loss: 0.009794 (epoch: 97, step: 0)
loss: 0.01076 (epoch: 97, step: 50)
loss: 0.01053 (epoch: 97, step: 100)
----- VALIDATING - EPOCH 97 -----
VAL loss: 0.01216 (epoch: 97, step: 0)
VAL loss: 0.01005 (epoch: 97, step: 50)
EPOCH IoU on VAL set:  45.49 %
----- TRAINING - EPOCH 98 -----
LEARNING RATE:  0.00032140030478246273
loss: 0.008002 (epoch: 98, step: 0)
loss: 0.0106 (epoch: 98, step: 50)
loss: 0.01048 (epoch: 98, step: 100)
----- VALIDATING - EPOCH 98 -----
VAL loss: 0.01269 (epoch: 98, step: 0)
VAL loss: 0.01003 (epoch: 98, step: 50)
EPOCH IoU on VAL set:  46.15 %
----- TRAINING - EPOCH 99 -----
LEARNING RATE:  0.0003195090954286887
loss: 0.008835 (epoch: 99, step: 0)
loss: 0.01043 (epoch: 99, step: 50)
loss: 0.01034 (epoch: 99, step: 100)
----- VALIDATING - EPOCH 99 -----
VAL loss: 0.01284 (epoch: 99, step: 0)
VAL loss: 0.0101 (epoch: 99, st

...

VAL loss: 0.007662 (epoch: 152, step: 50)
EPOCH IoU on VAL set:  49.43 %
----- TRAINING - EPOCH 153 -----
LEARNING RATE:  0.00021524222097152554
loss: 0.007143 (epoch: 153, step: 0)
loss: 0.008172 (epoch: 153, step: 50)
loss: 0.008225 (epoch: 153, step: 100)
----- VALIDATING - EPOCH 153 -----
VAL loss: 0.008418 (epoch: 153, step: 0)
VAL loss: 0.007408 (epoch: 153, step: 50)
EPOCH IoU on VAL set:  50.22 %
----- TRAINING - EPOCH 154 -----
LEARNING RATE:  0.00021326449437585992
loss: 0.008213 (epoch: 154, step: 0)
loss: 0.007695 (epoch: 154, step: 50)
loss: 0.007876 (epoch: 154, step: 100)
----- VALIDATING - EPOCH 154 -----
VAL loss: 0.008814 (epoch: 154, step: 0)
VAL loss: 0.007552 (epoch: 154, step: 50)
EPOCH IoU on VAL set:  51.28 %
----- TRAINING - EPOCH 155 -----
LEARNING RATE:  0.00021128472779749692
loss: 0.007417 (epoch: 155, step: 0)
loss: 0.008432 (epoch: 155, step: 50)
loss: 0.008266 (epoch: 155, step: 100)
----- VALIDATING - EPOCH 155 -----
VAL loss: 0.00992 (epoch: 155, step:

loss: 0.007583 (epoch: 176, step: 100)
----- VALIDATING - EPOCH 176 -----
VAL loss: 0.008825 (epoch: 176, step: 0)
VAL loss: 0.00688 (epoch: 176, step: 50)
EPOCH IoU on VAL set:  51.82 %
----- TRAINING - EPOCH 177 -----
LEARNING RATE:  0.00016716006998158345
loss: 0.008039 (epoch: 177, step: 0)
loss: 0.007231 (epoch: 177, step: 50)
loss: 0.007438 (epoch: 177, step: 100)
----- VALIDATING - EPOCH 177 -----
VAL loss: 0.008531 (epoch: 177, step: 0)
VAL loss: 0.007078 (epoch: 177, step: 50)
EPOCH IoU on VAL set:  51.80 %
----- TRAINING - EPOCH 178 -----
LEARNING RATE:  0.00016512566158078588
loss: 0.007094 (epoch: 178, step: 0)
loss: 0.007608 (epoch: 178, step: 50)
loss: 0.007489 (epoch: 178, step: 100)
----- VALIDATING - EPOCH 178 -----
VAL loss: 0.008654 (epoch: 178, step: 0)
VAL loss: 0.006866 (epoch: 178, step: 50)
EPOCH IoU on VAL set:  51.77 %
----- TRAINING - EPOCH 179 -----
LEARNING RATE:  0.00016308846431783376
loss: 0.01016 (epoch: 179, step: 0)
loss: 0.007527 (epoch: 179, step: 5

loss: 0.007 (epoch: 199, step: 50)
loss: 0.006976 (epoch: 199, step: 100)
----- VALIDATING - EPOCH 199 -----
VAL loss: 0.007856 (epoch: 199, step: 0)
VAL loss: 0.006204 (epoch: 199, step: 50)
EPOCH IoU on VAL set:  53.21 %
----- TRAINING - EPOCH 200 -----
LEARNING RATE:  0.00011957410943642218
loss: 0.006868 (epoch: 200, step: 0)
loss: 0.007159 (epoch: 200, step: 50)
loss: 0.007125 (epoch: 200, step: 100)
----- VALIDATING - EPOCH 200 -----
VAL loss: 0.007761 (epoch: 200, step: 0)
VAL loss: 0.006173 (epoch: 200, step: 50)
EPOCH IoU on VAL set:  52.78 %
----- TRAINING - EPOCH 201 -----
LEARNING RATE:  0.00011746189430880188
loss: 0.007607 (epoch: 201, step: 0)
loss: 0.006938 (epoch: 201, step: 50)
loss: 0.006994 (epoch: 201, step: 100)
----- VALIDATING - EPOCH 201 -----
VAL loss: 0.007576 (epoch: 201, step: 0)
VAL loss: 0.006118 (epoch: 201, step: 50)
EPOCH IoU on VAL set:  53.65 %
Saving model as best
save: ../save//model_best.pth (epoch: 201)
----- TRAINING - EPOCH 202 -----
LEARNING R

VAL loss: 0.005664 (epoch: 222, step: 50)
EPOCH IoU on VAL set:  54.26 %
----- TRAINING - EPOCH 223 -----
LEARNING RATE:  6.970536806419973e-05
loss: 0.00578 (epoch: 223, step: 0)
loss: 0.006559 (epoch: 223, step: 50)
loss: 0.006447 (epoch: 223, step: 100)
----- VALIDATING - EPOCH 223 -----
VAL loss: 0.007916 (epoch: 223, step: 0)
VAL loss: 0.005731 (epoch: 223, step: 50)
EPOCH IoU on VAL set:  54.31 %
----- TRAINING - EPOCH 224 -----
LEARNING RATE:  6.74607840325393e-05
loss: 0.006276 (epoch: 224, step: 0)
loss: 0.006164 (epoch: 224, step: 50)
loss: 0.006484 (epoch: 224, step: 100)
----- VALIDATING - EPOCH 224 -----
VAL loss: 0.007263 (epoch: 224, step: 0)
VAL loss: 0.0056 (epoch: 224, step: 50)
EPOCH IoU on VAL set:  54.28 %
----- TRAINING - EPOCH 225 -----
LEARNING RATE:  6.520786930851939e-05
loss: 0.007679 (epoch: 225, step: 0)
loss: 0.006376 (epoch: 225, step: 50)
loss: 0.006389 (epoch: 225, step: 100)
----- VALIDATING - EPOCH 225 -----
VAL loss: 0.007517 (epoch: 225, step: 0)
VA

loss: 0.006076 (epoch: 245, step: 50)
loss: 0.006094 (epoch: 245, step: 100)
----- VALIDATING - EPOCH 245 -----
VAL loss: 0.006733 (epoch: 245, step: 0)
VAL loss: 0.005331 (epoch: 245, step: 50)
EPOCH IoU on VAL set:  55.01 %
----- TRAINING - EPOCH 246 -----
LEARNING RATE:  1.4787576366283148e-05
loss: 0.00639 (epoch: 246, step: 0)
loss: 0.006069 (epoch: 246, step: 50)
loss: 0.006205 (epoch: 246, step: 100)
----- VALIDATING - EPOCH 246 -----
VAL loss: 0.006672 (epoch: 246, step: 0)
VAL loss: 0.00529 (epoch: 246, step: 50)
EPOCH IoU on VAL set:  55.07 %
----- TRAINING - EPOCH 247 -----
LEARNING RATE:  1.2097008581328884e-05
loss: 0.006778 (epoch: 247, step: 0)
loss: 0.005955 (epoch: 247, step: 50)
loss: 0.006101 (epoch: 247, step: 100)
----- VALIDATING - EPOCH 247 -----
VAL loss: 0.006785 (epoch: 247, step: 0)
VAL loss: 0.005246 (epoch: 247, step: 50)
EPOCH IoU on VAL set:  55.15 %
Saving model as best
save: ../save//model_best.pth (epoch: 247)
----- TRAINING - EPOCH 248 -----
LEARNING 